# Power View Simulator

In [1]:
import great_expectations as ge 
import pandas as pd
import warnings
import json

# Suprimir todos os warnings
warnings.filterwarnings("ignore")

In [2]:
# Lendo os dados
rawData = pd.read_csv('indicadoresCovid.csv')

# Criando o contexto de dados do Great Expectations
context = ge.get_context()

# Criando uma fonte de dados do Pandas
data_source = context.data_sources.add_pandas("pandas")
data_asset = data_source.add_dataframe_asset(name="pd dataframe asset")

batch_definition = data_asset.add_batch_definition_whole_dataframe("batch definition")
batch = batch_definition.get_batch(batch_parameters={"dataframe": rawData})

In [3]:
# Dimensão 1: Preenchimento
expectation_not_null_1 = ge.expectations.ExpectColumnValuesToNotBeNull(
    column="source_id",
    meta={"dimensao": "Preenchimento"}
)
expectation_not_null_2 = ge.expectations.ExpectColumnValuesToNotBeNull(
    column="dataNotificacao",
    meta={"dimensao": "Preenchimento"}
)
expectation_not_null_3 = ge.expectations.ExpectColumnValuesToNotBeNull(
    column="dataInicioSintomas",
    meta={"dimensao": "Preenchimento"}
)
expectation_not_null_4 = ge.expectations.ExpectColumnValuesToNotBeNull(
    column="uf",
    meta={"dimensao": "Preenchimento"}
)
expectation_not_null_5 = ge.expectations.ExpectColumnValuesToNotBeNull(
    column="idade",
    meta={"dimensao": "Preenchimento"}
)

# Dimensão 2: Padronização
expectation_Match_StrftimeFormat_1 = ge.expectations.ExpectColumnValuesToMatchStrftimeFormat(
    column="dataNotificacao", 
    strftime_format="%Y-%m-%d",
    meta={"dimensao": "Padronização"}
)
expectation_Match_StrftimeFormat_2 = ge.expectations.ExpectColumnValuesToMatchStrftimeFormat(
    column="dataInicioSintomas", 
    strftime_format="%Y-%m-%d",
    meta={"dimensao": "Padronização"}
)
expectation_Match_StrftimeFormat_3 = ge.expectations.ExpectColumnValuesToMatchStrftimeFormat(
    column="dataPrimeiraDose", 
    strftime_format="%Y-%m-%d",
    mostly=1.0,
    meta={"dimensao": "Padronização"}
)
expectation_Match_StrftimeFormat_4 = ge.expectations.ExpectColumnValuesToMatchStrftimeFormat(
    column="dataSegundaDose", 
    strftime_format="%Y-%m-%d",
    mostly=1.0,
    meta={"dimensao": "Padronização"}
)

expectation_column_values_to_be_in_set_1 = ge.expectations.ExpectColumnValuesToBeInSet(
    column="racaCor", 
    value_set=["AMARELA", "BRANCA", "IGNORADO", "INDIGENA", "PARDA", "PRETA"],
    meta={"dimensao": "Padronização"}
)
expectation_column_values_to_be_in_set_2 = ge.expectations.ExpectColumnValuesToBeInSet(
    column="sexo", 
    value_set=["Masculino", "Femenino", "Indefinido"],
    meta={"dimensao": "Padronização"}
)

# Validação de Laboratórios
laboratorios_validos = [
    "ASTRAZENECA/FIOCRUZ", 
    "JANSSEN", 
    "SINOVAC/BUTANTAN", 
    "PFIZER"
]
expectation_column_values_to_be_in_set_4 = ge.expectations.ExpectColumnValuesToBeInSet(
    column="codigoLaboratorioPrimeiraDose",
    value_set=laboratorios_validos,
    mostly=1.0,
    meta={"dimensao": "Padronização"}
)
expectation_column_values_to_be_in_set_5 = ge.expectations.ExpectColumnValuesToBeInSet(
    column="codigoLaboratorioSegundaDose",
    value_set=laboratorios_validos,
    mostly=1.0,
    meta={"dimensao": "Padronização"}
)

# Colunas com valores específicos em LETRAS MAIÚSCULAS
expectation_Values_to_Match_Regex_1 = ge.expectations.ExpectColumnValuesToMatchRegex(
    column="profissionalSaude",
    regex="^(Sim|Nao)$",
    meta={"dimensao": "Padronização"}
)
expectation_Values_to_Match_Regex_2 = ge.expectations.ExpectColumnValuesToMatchRegex(
    column="sintomas", 
    regex=r'^[A-Z\s]*$',
    mostly=1.0,
    meta={"dimensao": "Padronização"}
)
expectation_Values_to_Match_Regex_3 = ge.expectations.ExpectColumnValuesToMatchRegex(
    column="outrosSintomas", 
    regex=r'^[A-Z\s]*$',
    mostly=1.0,
    meta={"dimensao": "Padronização"}
)
expectation_Values_to_Match_Regex_4 = ge.expectations.ExpectColumnValuesToMatchRegex(
    column="Municipio", 
    regex=r'^[A-Z\s]*$',
    mostly=1.0,
    meta={"dimensao": "Padronização"}
)

expectation_Values_to_Match_Regex_5 = ge.expectations.ExpectColumnValuesToMatchRegex(
    column="idade",
    regex=r'^\d+$',
    meta={"dimensao": "Padronização"}
)

# Dimensão 3: Consistência
expectation_Values_to_Be_Between_1 = ge.expectations.ExpectColumnValuesToBeBetween(
    column="dataNotificacao",
    min_value="2020-01-04",
    max_value="2022-07-22",
    meta={"dimensao": "Consistência"}
)
expectation_Values_to_Be_Between_2 = ge.expectations.ExpectColumnValuesToBeBetween(
    column="dataInicioSintomas",
    min_value="2020-01-04",
    max_value="2022-07-22",
    meta={"dimensao": "Consistência"}
)
expectation_Values_to_Be_Between_3 = ge.expectations.ExpectColumnValuesToBeBetween(
    column="dataPrimeiraDose",
    min_value="2020-03-23",
    max_value="2022-07-22",
    mostly=1.0,
    meta={"dimensao": "Consistência"}
)
expectation_Values_to_Be_Between_4 = ge.expectations.ExpectColumnValuesToBeBetween(
    column="dataSegundaDose",
    min_value="2020-08-19",
    max_value="2022-07-22",
    mostly=1.0,
    meta={"dimensao": "Consistência"}
)
expectation_Values_to_Be_Between_5 = ge.expectations.ExpectColumnValuesToBeBetween(
    column="idade",
    min_value=0,
    max_value=110,
    strict_max=True,
    meta={"dimensao": "Consistência"}
)
expectation_column_pair_values_A_to_be_less_than_B = ge.expectations.ExpectColumnPairValuesAToBeGreaterThanB(
    column_A="dataPrimeiraDose",
    column_B="dataSegundaDose",
    mostly=1.0,
    meta={"dimensao": "Consistência"}
)

# Dimensão 4: Unicidade
expectation_column_values_to_be_unique = ge.expectations.ExpectColumnValuesToBeUnique(
    column="source_id",
    mostly=0.9,
    meta={"dimensao": "Unicidade"}
)

# Validação da UF
ufs_validas = [
    "AC","AL","AP","AM","BA","CE","DF","ES","GO","MA","MT","MS","MG",
    "PA","PB","PR","PE","PI","RJ","RN","RS","RO","RR","SC","SP","SE","TO"]

# Dimensão 5: Abrangência
expectation_column_values_to_be_in_set_3 = ge.expectations.ExpectColumnValuesToBeInSet(
    column="uf", 
    value_set=ufs_validas,
    meta={"dimensao": "Abrangência"}
)

In [4]:
# Criar o ExpectationSuite e adicionar todas as expectativas
suite = ge.core.ExpectationSuite(name="my_suite")

# -- Dimensão 1: Preenchimento
suite.add_expectation(expectation_not_null_1)
suite.add_expectation(expectation_not_null_2)
suite.add_expectation(expectation_not_null_3)
suite.add_expectation(expectation_not_null_4)
suite.add_expectation(expectation_not_null_5)

# -- Dimensão 2: Padronização
suite.add_expectation(expectation_Match_StrftimeFormat_1)
suite.add_expectation(expectation_Match_StrftimeFormat_2)
suite.add_expectation(expectation_Match_StrftimeFormat_3)
suite.add_expectation(expectation_Match_StrftimeFormat_4)

suite.add_expectation(expectation_column_values_to_be_in_set_1)
suite.add_expectation(expectation_column_values_to_be_in_set_2)
suite.add_expectation(expectation_column_values_to_be_in_set_4)

suite.add_expectation(expectation_Values_to_Match_Regex_1)
suite.add_expectation(expectation_Values_to_Match_Regex_2)
suite.add_expectation(expectation_Values_to_Match_Regex_3)
suite.add_expectation(expectation_Values_to_Match_Regex_4)
suite.add_expectation(expectation_Values_to_Match_Regex_5)

# -- Dimensão 3: Consistência
suite.add_expectation(expectation_Values_to_Be_Between_1)
suite.add_expectation(expectation_Values_to_Be_Between_2)
suite.add_expectation(expectation_Values_to_Be_Between_3)
suite.add_expectation(expectation_Values_to_Be_Between_4)
suite.add_expectation(expectation_Values_to_Be_Between_5)
suite.add_expectation(expectation_column_pair_values_A_to_be_less_than_B)

# -- Dimensão 4: Unicidade
suite.add_expectation(expectation_column_values_to_be_unique)

# -- Dimensão 5: Abrangência
suite.add_expectation(expectation_column_values_to_be_in_set_3)

# 3) Validar o ExpectationSuite no batch de dados
validation_results = batch.validate(suite)
print(validation_results["results"][-1])

# 4) Construir o DataFrame com a coluna 'dimensao'
results = validation_results["results"]

rows = []

for item in results:
    success = item.get("success")
    exp_config = item.get("expectation_config", {})
    exp_type = exp_config.get("type")
    kwargs = exp_config.get("kwargs", {})
    
    column_name = kwargs.get("column")
    column_A = kwargs.get("column_A")
    column_B = kwargs.get("column_B")
    batch_id = kwargs.get("batch_id")
    
    meta = exp_config.get("meta", {})
    dimensao = meta.get("dimensao", "Desconhecida")
    
    result_dict = item.get("result", {})
    unexpected_count = result_dict.get("unexpected_count")
    unexpected_percent = result_dict.get("unexpected_percent")
    partial_unexpected_list = result_dict.get("partial_unexpected_list")
    
    # Limitar o tamanho da lista para evitar warnings
    if isinstance(partial_unexpected_list, list):
        partial_unexpected_list = partial_unexpected_list[:10]  # Limita a 10 itens
    
    exception_info = item.get("exception_info", {})
    raised_exception = exception_info.get("raised_exception", False)
    exception_message = exception_info.get("exception_message")
    exception_traceback = exception_info.get("exception_traceback")

    row = {
        "success": success,
        "expectation_type": exp_type,
        "column_name": column_name,
        "column_A": column_A,
        "column_B": column_B,
        "batch_id": batch_id,
        "dimensao": dimensao,
        "unexpected_count": unexpected_count,
        "unexpected_percent": unexpected_percent,
        "partial_unexpected_list": partial_unexpected_list,
        "raised_exception": raised_exception,
        "exception_message": exception_message,
        "exception_traceback": exception_traceback}
    
    rows.append(row)

df = pd.DataFrame(rows)

# Exibir o DataFrame
display(df.sort_values(by="dimensao").reset_index(drop=True))

Calculating Metrics:   0%|          | 0/152 [00:00<?, ?it/s]

{
  "success": false,
  "expectation_config": {
    "type": "expect_column_pair_values_a_to_be_greater_than_b",
    "kwargs": {
      "batch_id": "pandas-pd dataframe asset",
      "column_A": "dataPrimeiraDose",
      "column_B": "dataSegundaDose"
    },
    "meta": {
      "dimensao": "Consist\u00eancia"
    }
  },
  "result": {
    "element_count": 10499,
    "unexpected_count": 5236,
    "unexpected_percent": 99.73333333333333,
    "partial_unexpected_list": [
      [
        "2020-03-23",
        null
      ],
      [
        "2026-01-05",
        null
      ],
      [
        "2020-08-13",
        "2020-08-27"
      ],
      [
        "2020-08-20",
        "2020-09-03"
      ],
      [
        "2020-08-27",
        null
      ],
      [
        "2020-08-28",
        "2020-09-11"
      ],
      [
        "2020-08-28",
        "2020-09-14"
      ],
      [
        "2020-08-28",
        "2020-09-11"
      ],
      [
        "2020-08-31",
        "2020-09-14"
      ],
      [
       

,success,expectation_type,column_name,column_A,column_B,batch_id,dimensao,unexpected_count,unexpected_percent,partial_unexpected_list,raised_exception,exception_message,exception_traceback
0,True,expect_column_values_to_be_in_set,uf,None,None,pandas-pd dataframe asset,Abrangência,0.0,0.000000,[],False,None,None
1,False,expect_column_values_to_be_between,dataNotificacao,None,None,pandas-pd dataframe asset,Consistência,NaN,NaN,None,False,None,None
2,False,expect_column_values_to_be_between,idade,None,None,pandas-pd dataframe asset,Consistência,7.0,0.066698,"[132.0, 129.0, 150.0, 144.0, 138.0, 122.0, 143.0]",False,None,None
3,False,expect_column_pair_values_a_to_be_greater_than_b,None,dataPrimeiraDose,dataSegundaDose,pandas-pd dataframe asset,Consistência,5236.0,99.733333,"[(2020-03-23, nan), (2026-01-05, nan), (2020-0...",False,None,None
4,False,expect_column_values_to_be_between,dataInicioSintomas,None,None,pandas-pd dataframe asset,Consistência,NaN,NaN,None,False,None,None
5,False,expect_column_values_to_be_between,dataPrimeiraDose,None,None,pandas-pd dataframe asset,Consistência,NaN,NaN,None,False,None,None
6,False,expect_column_values_to_be_between,dataSegundaDose,None,None,pandas-pd dataframe asset,Consistência,NaN,NaN,None,False,None,None
7,False,expect_column_values_to_match_regex,outrosSintomas,None,None,pandas-pd dataframe asset,Padronização,2535.0,60.099573,"[TERMINAL DE INTEGRAÇÃO, TERMINAL RODOVIA´RIO,...",False,None,None
8,False,expect_column_values_to_match_regex,sintomas,None,None,pandas-pd dataframe asset,Padronização,10467.0,100.000000,"[Tosse, Febre, Tosse, Dor de Garganta, Tosse, ...",False,None,None
9,True,expect_column_values_to_match_strftime_format,dataNotificacao,None,None,pandas-pd dataframe asset,Padronização,0.0,0.000000,[],False,None,None
